In [1]:
# Set up
import pandas as pd
import random
SEED = 20
random.seed(SEED)

# Load all image paths
dataset_path = '../Datasets/CIFAR10'
data_df = pd.read_csv(dataset_path + '/data.csv')

# Dataset Partition

In [2]:
# Split test dataset and private (data owner) dataset
test_dataset = data_df.groupby('label').sample(frac=.2) 
private_dataset = data_df.iloc[[i for i in data_df.index if i not in test_dataset.index]]

In [3]:
# Generate a random hundreds, default: between 500 ~ 3000
def get_random_hundreds(low=500, high=3000):
    return round(random.randint(low//100, high//100)) * 100

In [4]:
# Generate data owner datasets by labels
def generate_data_owner_datasets(labels):
    label_count_info = {label:get_random_hundreds() for label in labels}
    return label_count_info

In [5]:
# Set data owner label information
data_owner_label_info = {
    'A': ('trainer', 'plane, car, ship, boat, truck, other'),
    'B': ('trainer', 'truck, car, other'),
    'C': ('trainer', 'plane, truck, ship, other'),
    'D': ('trainer', 'ship, plane'),
    'E': ('trainer', 'plane, car, ship, other'),  # (overlapping dataset)
    'F': ('trainer', 'cat, dog, other'),  # (non existing labels)
    'G': ('trainer', 'truck, other'),  # (drop below baseline models)
    'T1': ('trainer', 'truck, car'),  # (all labels below baseline)
    'T2': ('trainer', 'plane, boat'),  # (one label below baseline)
    'T3': ('trainer', 'cat, dog'),  # (large model)
    'X': ('client', 'truck'),
    'Y': ('client', 'truck'), # (select models above baseline)
    'Z': ('client', 'plane, ship'), # (multiple labels)
    'T4': ('client', 'horse') # (non existing)
}
data_owner_label_info = {k:(v[0], v[1].split(', ')) for k,v in data_owner_label_info.items()}
data_owner_label_count_info = {k:{label:get_random_hundreds() for label in v[1]} for k,v in data_owner_label_info.items()}
all_labels = set()
for k,v in data_owner_label_count_info.items():
    all_labels.update(list(v.keys()))
    print(k, '\t', '\t'.join([f'{i[0]}({i[1]})' for i in v.items()]))

all_labels.remove('other')
print('All Labels:', ', '.join(list(all_labels)))

A 	 plane(2800)	car(2600)	ship(3000)	boat(2900)	truck(900)	other(1300)
B 	 truck(2600)	car(2500)	other(800)
C 	 plane(1500)	truck(2300)	ship(1000)	other(500)
D 	 ship(1800)	plane(1800)
E 	 plane(700)	car(800)	ship(900)	other(1500)
F 	 cat(2000)	dog(2300)	other(1900)
G 	 truck(1800)	other(1100)
T1 	 truck(1100)	car(1500)
T2 	 plane(2500)	boat(2600)
T3 	 cat(1500)	dog(1500)
X 	 truck(1800)
Y 	 truck(700)
Z 	 plane(2500)	ship(2100)
T4 	 horse(3000)
All Labels: car, plane, cat, horse, truck, dog, ship, boat


In [6]:
# Generate a list with the given length
def generate_list_by_sum(m, n):
    arr = [0] * m;
    for i in range(n):
        arr[random.randint(0, m-1)] += 1
    return arr

# update the count for other labels
data_owner_label_count_full_info = {}
for k, v in data_owner_label_count_info.items():
    other_labels = all_labels.difference(set(v.keys()))
    data_owner_label_count_full_info[k] = v
    if 'other' in v:
        total_other_labels = v['other']
        del data_owner_label_count_full_info[k]['other']
        for label, count in zip(other_labels, generate_list_by_sum(len(other_labels), total_other_labels)):
            data_owner_label_count_full_info[k][label] = f'{count}*'
    
    # Set count of extra labels to 0
    extra_labels = all_labels.difference(set(data_owner_label_count_full_info[k].keys()))
    for label in extra_labels:
        data_owner_label_count_full_info[k][label] = '0*'
    for label in v.keys():
        data_owner_label_count_full_info[k][label] = f'{data_owner_label_count_full_info[k][label]}'

In [7]:
data_owner_info_df = pd.DataFrame(data_owner_label_count_full_info).T
data_owner_info_df

,plane,car,ship,boat,truck,dog,horse,cat
A,2800,2600,3000,2900,900,466*,398*,436*
B,139*,2500,127*,138*,2600,132*,127*,137*
C,1500,110*,1000,84*,2300,96*,107*,103*
D,1800,0*,1800,0*,0*,0*,0*,0*
E,700,800,900,287*,291*,318*,274*,330*
F,341*,307*,327*,315*,296*,2300,314*,2000
G,136*,149*,184*,157*,1800,165*,149*,160*
T1,0*,1500,0*,0*,1100,0*,0*,0*
T2,2500,0*,0*,2600,0*,0*,0*,0*
T3,0*,0*,0*,0*,0*,1500,0*,1500
